In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv();

In [141]:
import json
import logging
import os
import re
import sys
from functools import partial
from itertools import chain
from pathlib import Path
from uuid import uuid4

import bhutils
import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import clarion
import QuantLib as ql
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display
from pandas.tseries.offsets import BDay
from scipy.interpolate import CubicSpline
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from blp.blp import BlpParser

from gioutils import ezutils as ez
from gioutils.gui.pandas import millifyp as millify
from gioutils.blpw import BlpParser, BlpQuery
from gioutils.clarion_ext import super_clarion, Runner
from gioutils.tools import parse_swap_periods
from gioutils.ezutils import (
    bh,
    bhs,
    get_id,
    get_price_info,
)

from gioutils.utils import (
    get_bday,
    get_next_n,
    parse_offset,
    today,
    get_generic_future, 
    get_bbg_fut_chain_ticker,
    get_root
)
pd.options.plotting.backend = "plotly"
pd.options.display.float_format = partial(millify, pct=None, decimal=4)

In [4]:
clarion.server_login()
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()
# ba = BHSysApi(password="A&3jg5Aj")

2023-02-28 10:41:57,177 - Authenticating...
2023-02-28 10:41:57,871 - Successfully authenticated


In [5]:
super_clarion(clarion)

In [53]:
runner = Runner(clarion)

### Metrics

In [148]:
clarion_metrics = dict(
    pv = clarion.metric('pv', 'PresentValue', 'Any'),
    local_pv = clarion.metric('local_pv', 'LocalPresentValue', 'Any'),
    # currency = clarion.metric('currency', 'PayCurrency', 'Any'),
    strike = clarion.metric('strike', 'Strike', 'Any'),
    notional = clarion.metric('notional', 'Notional', 'Any'),
    expiry = clarion.metric('expiry', 'ExpiryDate', 'Any'),
    maturity = clarion.metric('maturity_date', 'MaturityDate', 'Rates', {'Type': 'Bumped'}),
    start = clarion.metric('start_date', 'StartDate', 'Rates', {'Type': 'Bumped'}),
    fwd_rate = clarion.metric('fwd_rate', 'Forward', 'Rates'),
    delta = clarion.metric('delta', 'Delta', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'}),
    gamma = clarion.metric('gamma', 'Gamma', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'}),
    theta = clarion.metric('theta', 'OptionTheta', 'Rates', {'Period': '1'}),
    price_cents = clarion.metric('price_cents', 'Price', 'Rates', {'Unit': 'Cents'}),
    price_bps = clarion.metric('price_bps', 'Price', 'Rates', {'Unit': 'BasisPoints'}),
    price_pct = clarion.metric('price_pct', 'Price', 'Equity', {'Unit': 'PercentLocal'}),
    price_unit = clarion.metric('price_unit', 'Price', 'Equity', {'Unit': 'Local'}),
    spot = clarion.metric('spot', 'Spot', 'Rates'),
    # vega = clarion.metric('vega', 'Vega', 'Rates', {'Period': 'Daily'}),
    # atm_vol_day = clarion.metric('atm_vol_day', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Daily'}),
    atm_vol_ann = clarion.metric('atm_vol_ann', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Annual'}),
    # strike_vol_day = clarion.metric('strike_vol_day', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Daily'}),
    strike_vol_ann = clarion.metric('strike_vol_ann', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Annual'}),
    # carry_day = clarion.metric('carry_day', 'Carry', 'Credit', {'Horizon': '1d'}),
)


In [6]:
future_months_codes = [
    'F',
    'G',
    'H',
    'J',
    'K',
    'M',
    'N',
    'Q',
    'U',
    'V',
    'X',
    'Z'
]

future_order = [f"{month}{year}" for year in range(0, 12 + 1) for month in future_months_codes]

In [128]:
mid_curve_map = {
    'sfr': [
        # 'sra', 
        # 'srr', 
        # 'srw', 
        '0q', 
        '2q', 
        '3q',
        '4q',
    ],
    'er': [
        '0r', 
        '2r', 
        '3r',
        '4r'
    ]
}

fixing_map = {
    'sfr': 'SOFRRATE Index',
    'er': 'EUR003M Index',
    'rx': 'GTDEM10Y Govt'

}

clarion_ticker_map = {
    'rx': 'BUND EUX',
    'ty': ''
}

clarion_call_put_map = {
    'c': 'Call',
    'p': 'Put'
}

In [43]:
def get_underlying_ticker(underlying, contract, yk=None):
    yk = yk or 'Comdty'
    return f"{underlying.upper()}{contract.upper() if isinstance(contract, str) else contract} {yk.title()}"

In [90]:
def get_fut_chain(underlying, contracts=None, yk=None, spot_yield=True):
    contracts = contracts or range(1, 16 + 1)
    yk = yk or 'Comdty'
    securities = [get_underlying_ticker(underlying, contract, yk) for contract in contracts]
    tickers = [*(bq.bdp(securities, {"FUT_CUR_GEN_TICKER": 'ticker'})['ticker'] + ' ' + yk)]
    ticker_map = {contract: ticker for contract, ticker in zip(contracts, tickers)}
    prices = bq.bdp([fixing_map[underlying.lower()], *tickers], {"px_last": 'price', "LAST_TRADEABLE_DT": 'expiry'})
    prices['expiry_code'] = prices['security'].str.replace(underlying.upper(), '').str.split(' ').str[0]
    prices['expiry_code_full'] = prices['expiry_code'].str[0] + str(today().year)[-2] + prices['expiry_code'].str[1]
    prices.iloc[0, -1] = np.nan
    prices.loc[prices['security'] == fixing_map[underlying.lower()], 'expiry'] = today()
    if spot_yield:
        prices.loc[prices['security'] == fixing_map[underlying.lower()], 'price'] = 100 - prices.loc[prices['security'] == fixing_map[underlying.lower()], 'price']
    days = (prices['expiry'] - today()).dt.days
    prices['days'] = days
    return prices

In [96]:
def get_opt_chain_and_info(underlying):
    chain_ = bq.bds(underlying, 'OPT_CHAIN')
    if chain_.empty:
        return
    chain_ = chain_.squeeze().rename('chain')
    chain = chain_.str.replace(r'\s+', ' ', regex=True).str.split(' ')
    chain = pd.DataFrame.from_dict(dict(zip(chain.index, chain.values))).T.set_axis(['code', 'strike', 'yk'], axis=1)
    chain['call_put'] = chain['code'].str[-1]
    chain['code'] = chain['code'].str[:-1]
    chain['strike'] = chain['strike'].astype(float)
    chain['orig'] = chain_
    
    opt_chain = chain.drop_duplicates('code').set_index('code')['orig']
    opt_expiry = bq.bdp(opt_chain, {'LAST_TRADEABLE_DT': 'expiry', 'CRNCY': 'ccy'}).rename({'security': 'orig'}, axis=1)
    
    chain['ccy'] = opt_expiry['ccy'].drop_duplicates().squeeze()
    
    def get_strike_increment(df):
        return df['strike'].head(2).diff().dropna().squeeze()
    increments = chain.groupby('code', sort=False).apply(get_strike_increment).rename('inc')
    opt_info = opt_expiry.merge(opt_chain.reset_index(), on='orig').merge(increments, on='code')
    opt_info['expiry_code'] = opt_info['code'].str[-2:]
    opt_info['expiry_code_full'] = opt_info['expiry_code'].str[0] + str(today().year)[-2] + opt_info['expiry_code'].str[1]
    
    return chain, opt_info

In [ ]:
str(today().year)[-2]

'2'

In [126]:
underlying = 'rx'
opt_type = 'P'
contracts = [2]
underlying_ticker = get_underlying_ticker('rx', 'm3')

In [111]:
fut_prices = get_fut_chain(underlying, contracts, spot_yield=False)

In [109]:
opt_chain, opt_info = get_opt_chain_and_info(underlying_ticker)

In [127]:
opt_chain.query(f'code == "{opt["code"]}" and call_put == "{opt_type}"')

code  strike      yk call_put                   orig  ccy
0   RXJ3   115.0  Comdty        P    RXJ3P    115 Comdty  EUR
1   RXJ3   116.0  Comdty        P    RXJ3P    116 Comdty  EUR
2   RXJ3   117.0  Comdty        P    RXJ3P    117 Comdty  EUR
3   RXJ3   118.0  Comdty        P    RXJ3P    118 Comdty  EUR
4   RXJ3   119.0  Comdty        P    RXJ3P    119 Comdty  EUR
..   ...     ...     ...      ...                    ...  ...
58  RXJ3   147.0  Comdty        P    RXJ3P    147 Comdty  EUR
59  RXJ3   147.5  Comdty        P  RXJ3P    147.5 Comdty  EUR
60  RXJ3   148.0  Comdty        P    RXJ3P    148 Comdty  EUR
61  RXJ3   148.5  Comdty        P  RXJ3P    148.5 Comdty  EUR
62  RXJ3   149.0  Comdty        P    RXJ3P    149 Comdty  EUR

[63 rows x 6 columns]

In [132]:
instruments = [] 

for i, fut in fut_prices.iloc[1:, :].iterrows():
    for j, opt in opt_info.iterrows():
        options = opt_chain.query(f'code == "{opt["code"]}" and call_put == "{opt_type}"')
        for k, opt_strike in options.iterrows():
            instrument = i = clarion.instruments.RatesBondFutureOption(
                currency=opt['ccy'],
                instrument_name=clarion_ticker_map.get(underlying.lower()).upper(),
                expiry=opt['expiry_code_full'].upper(),
                tenor=fut['expiry_code_full'].upper(),
                strike=opt_strike['strike'],
                option_type=clarion_call_put_map[opt_strike['call_put'].lower()],
                notional=100,
            )
            instruments.append(instrument)

In [149]:
data = runner.run(
    metrics=[*clarion_metrics.values()], 
    data_list=instruments
)

2023-02-28 12:14:48,878 - Requesting data
2023-02-28 12:15:14,900 - Request complete
2023-02-28 12:15:18,116 - Response Count Summary - DFRAME:100, IS_ERROR:200
2023-02-28 12:15:18,119 - Requesting data
2023-02-28 12:15:18,696 - Request complete
2023-02-28 12:15:21,954 - Response Count Summary - DFRAME:100, IS_ERROR:200
2023-02-28 12:15:21,957 - Requesting data
2023-02-28 12:15:22,539 - Request complete
2023-02-28 12:15:22,704 - Response Count Summary - DFRAME:5, IS_ERROR:10


In [150]:
bp_shift = 10 

In [161]:
data['pv_scenario_chg'] = data['delta'] * bp_shift + 0.5 * data['gamma'] * (bp_shift ** 2)

In [162]:
data['pv_scenario_pct'] = data['pv_scenario_chg'] / data['pv']

In [163]:
cols = [
    'description',
    'pv',
    'pv_scenario_chg',
    'pv_scenario_pct',
    'delta',
    'gamma',
    
]

In [193]:
data.loc[: , cols].sort_values('pv_scenario_pct', ascending=False).to_clipboard()

In [146]:
data['pv']

0      24.0912
1      40.6102
2      68.9055
3     117.6402
4     201.9776
        ...   
200      1.64m
201      1.70m
202      1.75m
203      1.80m
204      1.85m
Name: pv, Length: 205, dtype: float64

In [195]:
fut_prices

security    price     expiry expiry_code expiry_code_full  days
0  GTDEM10Y Govt  96.9610 2023-02-28    GTDEM10Y              NaN     0
1    RXM3 Comdty 131.7600 2023-06-08          M3              M23   100

In [6]:
### check metrics
pd.concat(clarion.metrics()).to_clipboard()

In [218]:
usd_curve_shift = clarion.create_shift('RatesCurveShift', 'relative', 0.001, {'curveFamily': 'USD'})
eur_vol_shift = clarion.create_shift('RatesVolShift', 'relative', 0.001, {'curveFamily': 'EUR'})
eur_curve_shift = clarion.create_shift('RatesCurveShift', 'relative', 0.001, {'curveFamily': 'EUR'})
# scenario_rates = clarion.create_scenario('ScenRates', [usd_curve_shift, eur_curve_shift])
scenario_rates = clarion.create_scenario('ScenRates', [eur_curve_shift, ])

In [202]:
response['DFRAME']

Empty DataFrame
Columns: []
Index: []

In [212]:
i = clarion.instruments.RatesSwaption(
            currency='EUR',
            instrument_name='EURIBOR A 6M',
            expiry='1y',
            tenor='10y',
            strike='a',
            swaption_type='Pay',
            notional=100e6,
            settle_type='Physical Cleared LCH',
            collateral_currency='USD',
            pay_settle='Spot',
            #id='eur_9m_bear_flat'
        ).trade

In [219]:
#Run the fx scenario on the fx positions
response = clarion.risks.run(
    metrics=[clarion_metrics[metric] for metric in ['delta', 'pv', 'local_pv']], 
    data_list=clarion.create_scenario_population('rates', trade_list=[i]),
    scenarios=[scenario_rates], 
    # asof_date='2022-09-01', 
    market_mode='live'
)
if response['ERROR']:
    print(response['ERROR'])
else:
    display(response['DFRAME'])

2023-02-28 13:30:10,853 - Requesting data
2023-02-28 13:30:19,687 - Request complete
2023-02-28 13:30:19,692 - Response Count Summary - DFRAME:1


scenario_id                                              id  \
0   ScenRates  ratesswaption_21b4ce2ab63944598ed135a60ac0fd25   

                  type   Notional             TradeDate  TradePrice  \
0  PriceableIdentifier  100000000  2023-02-28T00:00:00Z           0   

  PayReceive             StartDate          MaturityDate Clearer  \
0      Payer  2024-03-01T00:00:00Z  2034-03-01T00:00:00Z     LCH   

     Underlying  Strike                           Description  \
0  EURIBOR A 6M  3.1906  EUR EURIBOR A 6M 1Yx10Y 3.1906 Payer   

             ExpiryDate LocalCurrency  local_pv    pv  delta_EUR  delta_USD  
0  2024-02-28T00:00:00Z           EUR     3.83m 4.07m     41.88k          0

In [ ]:
clarion.create_scenario_population('rates', trade_list[i.trade for in in instruments][:5])

In [189]:
#Run the fx scenario on the fx positions
response = clarion.risks.run(
    metrics=[clarion_metrics[metric] for metric in ['delta', 'pv', 'local_pv']], 
    data_list=[i][:5],
    scenarios=[scenario_rates], 
    # asof_date='2022-09-01', 
    market_mode='live'
)
if response['ERROR']:
    print(response['ERROR'])
else:
    display(response['DFRAME'])

2023-02-28 12:36:21,699 - Requesting data
2023-02-28 12:36:25,827 - Request complete
2023-02-28 12:36:25,831 - Response Count Summary - DFRAME:0


Empty DataFrame
Columns: []
Index: []

In [179]:
instruments[0].trade.instrument.instrument_data

{'Currency': 'EUR',
 'InstrumentName': 'BUND EUX',
 'Expiry': 'J23',
 'Tenor': 'M23',
 'Strike': 115.0,
 'OptionType': 'Put',
 'Notional': 100}

In [181]:
i = clarion.create_trade(
    id='a',
    trade_type='RatesBondFutureOption',
    instrument_data=instruments[0].trade.instrument.instrument_data,
    asset_class='Rates'
)

In [191]:
i

id: a, tradeType: RatesBondFutureOption, {'Currency': 'EUR', 'InstrumentName': 'BUND EUX', 'Expiry': 'J23', 'Tenor': 'M23', 'Strike': 115.0, 'OptionType': 'Put', 'Notional': 100}